# End-to-End Force Prediction
This notebook illustrates the complete workflow starting with raw TFM images and ending with a U-Net prediction of traction forces.

## 1. TFM processing
We first register bead images, compute displacements, calculate traction stresses and create masks.

In [ ]:
from TFM_main.TFM_Image_registration import TFM_Image_registration
from TFM_main.TFM_displacement_tools import TFM_optical_flow
from TFM_main.TFM_tools import TFM_calculation, cellmask_threshold
import skimage.io as io, numpy as np, os

In [ ]:
# Align bead stack to reference and compute optical flow
TFM_Image_registration(flatfield_correct=False)
TFM_optical_flow()
# Convert displacements into traction forces
TFM_calculation(regparam=1e-16)
# Threshold the fluorescence channel to create masks
cellmask, forcemask, _ = cellmask_threshold('zyxin_registered.tif', plot_figure=False)

## 2. Convert to dataset format
Here we combine the displacement fields, traction forces and fluorescence images into the `[ux, uy, Fx, Fy, mask, forcemask, zyxin, actin]` stack expected by `CellDataset`.

In [ ]:
zyx_stack = io.imread('zyxin_registered.tif')
act_stack = io.imread('actin_registered.tif')
os.makedirs('dataset/cell_0', exist_ok=True)
for i in range(zyx_stack.shape[0]):
    ux = io.imread(f'displacement_files/disp_u_{i:03d}.tif')
    uy = io.imread(f'displacement_files/disp_v_{i:03d}.tif')
    fx = io.imread(f'traction_files/fx_{i+1:03d}.tif')
    fy = io.imread(f'traction_files/fy_{i+1:03d}.tif')
    frame = np.stack([ux, uy, fx, fy, cellmask[i], forcemask[i], zyx_stack[i], act_stack[i]])
    np.save(f'dataset/cell_0/frame_{i}.npy', frame)

## 3. Train or load the U-Net
After generating the `.npy` files, use `train_unet.ipynb` to train on your dataset or `load_trained_unet.ipynb` to load a pre-trained model and make predictions.